Перед нами стоит задача предсказания числового значения на основе заданного текста. По заголовку мы должны предсказать его числовой рейтинг от 0.0 до 10.0 с точностью 0.1. 

В качестве библиотеки, реализующей модель русского языка, мы используем spaCy (в первую очередь из соображений скорости).

- Лемматизация
- Составление триграмм
- Категоризация отрезков текста

In [6]:
import pickle
import numpy as np
import pandas as pd

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# https://spacy.io/models/ru
import spacy
nlp = spacy.load('ru_core_news_sm')

DATASETS_PATH = '/home/leo/DATASETS'

with open('../sources.pickle', 'rb') as f:
    sources = pickle.load(f)
    
dfs = dict()
for source in sources:
    dfs[source] = pd.read_csv(f"{DATASETS_PATH}/{source}.csv", index_col=0, parse_dates=['post_time', 'parse_time'])
    dfs[source]['source'] = source

df = pd.concat(dfs[key] for key in dfs)
    
# удаляем дубликаты
df = df.drop_duplicates()    

# удаляем закрытые записи без просмотров
#df = df.drop(df[df.views_num == np.NINF].index)

df = pd.concat([dfs[key] for key in dfs])
df.views_num = df.views_num.apply(lambda x: int(''.join(filter(str.isdigit, str(x)))))
df.views_num = np.log(df.views_num)/np.log(df.views_num.max())
df = df.drop(df[df.views_num == np.NINF].index)

/home/leo/.local/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


# Оценка заголовка
Есть две модели: модель генерации заголовка и модель оценки заголовка. Начнем с модели оценки.

In [18]:
Xy = df[['title', 'views_num']]
X = Xy.title
y = Xy.views_num
Xy.sort_values(by='views_num', ascending=False).head(50)

,title,views_num
https://habr.com/ru/post/491974,Коронавирус: почему надо действовать прямо сейчас,1.000000
https://habr.com/ru/post/134892,Сети для самых маленьких. Часть нулевая. Плани...,0.923937
https://habr.com/ru/post/109944,Пишем своё первое приложение на Android,0.922720
https://habr.com/ru/post/54999,Скрытые смайлы в Skype,0.922335
https://habr.com/ru/post/250621,300 потрясающих бесплатных сервисов,0.917149
https://habr.com/ru/post/143834,Взломать Wi-Fi за 10 часов,0.913312
https://habr.com/ru/post/150302,Учим Python качественно,0.910403
https://habr.com/ru/post/149447,Wi-Fi: неочевидные нюансы (на примере домашней...,0.907601
https://habr.com/ru/post/488490,Ликбез по респираторам. Помогает ли респиратор...,0.906573
https://habr.com/ru/post/129664,"Еще раз про IP-адреса, маски подсетей и вообще",0.903128


In [31]:
for token in nlp(X.iloc[0]):
    print(token, token.lemma_)

7 7
мобильных мобильный
приложений приложение
для для
изучения изучение
программирования программирование


In [203]:
# def load_text(source, filename):
#     try:
#         with open(f'{DATASETS_PATH}/{source}/{filename}', 'r') as f:
#             return f.read()
#     except FileNotFoundError:
#         return ''
    
# df['full_text'] = df.apply(lambda x: load_text(x.source, x.filename), axis=1)